In [2]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = 'True'

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import logging

In [5]:
# Menyesuaikan level logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

In [6]:
# Input Dataset
df = pd.read_csv('Dataset_Manual_stemming.csv', encoding='latin1', delimiter=';')
df.head()

,No,Type,Mentions,Date,Media,Sentiment,Author,Followers,Retweeted,Favourited,Mentions1,Sentiment1,stemmed_text
0,1,rt,RT @LANGKAHANIES: Jangan ada intervensi politi...,31/05/2023 23:59,Twitter,Positive,@YTrigusmintara (ManusiaBebas),19.0,64.0,0.0,jangan ada intervensi politik penjegalan pilpr...,negatif,jangan ada intervensi politik jegal pilpres 20...
1,2,rt,RT @triwul82: Sejumlah perwakilan Koalisi Peru...,31/05/2023 23:59,Twitter,Negative,@INA_NKRI (100% Indonesia ÃÂÃÂÃÂÃÂ°ÃÂ...,1250.0,83.0,0.0,sejumlah perwakilan koalisi perubahan yang men...,positif,jumlah wakil koalisi ubah yang usung anies bag...
2,3,rt,RT @ajengcute16__: Merupakan Open Legal Policy...,31/05/2023 23:59,Twitter,Positive,@sri08054 (Sri anies),356.0,50.0,0.0,merupakan open legal policy perludem sangat be...,negatif,rupa open legal policy perludem sangat bahaya ...
3,4,rt,RT @Jatayu_45: JOKOWI HARUS MUNDUR DARI JABATA...,31/05/2023 23:59,Twitter,Neutral,@wongedan1708 (BAGong Modern),16.0,108.0,0.0,jokowi harus mundur dari jabatan presiden kala...,negatif,jokowi harus mundur dari jabat presiden kalau ...
4,5,mention,"Langkahi Presiden dan DPR, Demokrat: Bukan Wew...",31/05/2023 23:59,Twitter,Negative,@Simanjunta9Nico (Nico Simanjuntak),650.0,0.0,0.0,langkahi presiden dan dpr demokrat bukan wewen...,negatif,langkah presiden dan dpr demokrat bukan wewena...


In [7]:
# Pastikan data teks berupa string
df['stemmed_text'] = df['stemmed_text'].fillna('').astype(str)

In [8]:
# Mengkodekan label
label_encoder = LabelEncoder()
df['Sentiment1'] = label_encoder.fit_transform(df['Sentiment1'])

In [9]:
# Siapkan tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [10]:
# Konversikan data ke BERT InputExamples
def convert_data_to_examples(data, data_column, label_column):
    return data.apply(lambda x: InputExample(guid=None,
                                             text_a=x[data_column],
                                             text_b=None,
                                             label=x[label_column]), axis=1)


In [11]:
# Ubah InputExamples menjadi InputFeatures
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = []
    for e in examples:
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            return_token_type_ids=True,
            return_attention_mask=True,
            truncation=True
        )
        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
                                                     input_dict["token_type_ids"],
                                                     input_dict["attention_mask"])

        features.append(
            InputFeatures(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label)
        )

    def gen():
        for f in features:
            yield (
                {
                    'input_ids': f.input_ids,
                    'attention_mask': f.attention_mask,
                    'token_type_ids': f.token_type_ids
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({'input_ids': tf.int32, 'attention_mask': tf.int32, 'token_type_ids': tf.int32}, tf.int64),
        ({'input_ids': tf.TensorShape([None]), 'attention_mask': tf.TensorShape([None]), 'token_type_ids': tf.TensorShape([None])}, tf.TensorShape([])),
    )

In [15]:
# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracy_list = []

for fold, (train_index, test_index) in enumerate(kf.split(df), 1):
    train = df.iloc[train_index]
    test = df.iloc[test_index]
    
    # Convert to BERT input format
    train_InputExamples = convert_data_to_examples(train, 'stemmed_text', 'Sentiment1')
    test_InputExamples = convert_data_to_examples(test, 'stemmed_text', 'Sentiment1')
    
    train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
    train_data = train_data.shuffle(100).batch(32).repeat()  # Menambahkan .repeat()
    
    test_data = convert_examples_to_tf_dataset(list(test_InputExamples), tokenizer)
    test_data = test_data.batch(32)
    
    # Load BERT model
    model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
    
    # Compile the model with run_eagerly=True
    from tensorflow.keras.optimizers.legacy import Adam  
    optimizer = Adam(learning_rate=2e-5, epsilon=1e-08, clipnorm=1.0)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric], run_eagerly=True)
    
    # Determine the number of steps per epoch and validation steps
    train_steps_per_epoch = len(train) // 32
    validation_steps = len(test) // 32
    
    # Train the model
    model.fit(train_data, epochs=20, steps_per_epoch=train_steps_per_epoch, validation_data=test_data, validation_steps=validation_steps)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
124/124 [==============================] - 1800s 15s/step - loss: 0.8045 - accuracy: 0.6439 - val_loss: 0.5510 - val_accuracy: 0.7853
Epoch 2/20
124/124 [==============================] - 1743s 14s/step - loss: 0.4535 - accuracy: 0.8361 - val_loss: 0.3567 - val_accuracy: 0.8760
Epoch 3/20
124/124 [==============================] - 1857s 15s/step - loss: 0.3064 - accuracy: 0.8900 - val_loss: 0.3295 - val_accuracy: 0.8831
Epoch 4/20
124/124 [==============================] - 1782s 14s/step - loss: 0.2200 - accuracy: 0.9289 - val_loss: 0.3447 - val_accuracy: 0.8962
Epoch 5/20
124/124 [==============================] - 1785s 14s/step - loss: 0.1535 - accuracy: 0.9528 - val_loss: 0.3703 - val_accuracy: 0.8921
Epoch 6/20
124/124 [==============================] - 1778s 14s/step - loss: 0.1156 - accuracy: 0.9654 - val_loss: 0.4212 - val_accuracy: 0.8992
Epoch 7/20
124/124 [==============================] - 1913s 15s/step - loss: 0.1081 - accuracy: 0.9677 - val_loss: 0.4342 - val_ac

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
124/124 [==============================] - 1813s 15s/step - loss: 0.7586 - accuracy: 0.6910 - val_loss: 0.5359 - val_accuracy: 0.8044
Epoch 2/20
124/124 [==============================] - 1725s 14s/step - loss: 0.4441 - accuracy: 0.8348 - val_loss: 0.4307 - val_accuracy: 0.8367
Epoch 3/20
124/124 [==============================] - 1823s 15s/step - loss: 0.2971 - accuracy: 0.8956 - val_loss: 0.3818 - val_accuracy: 0.8619
Epoch 4/20
124/124 [==============================] - 1782s 14s/step - loss: 0.1998 - accuracy: 0.9284 - val_loss: 0.4025 - val_accuracy: 0.8760
Epoch 5/20
124/124 [==============================] - 1841s 15s/step - loss: 0.1458 - accuracy: 0.9546 - val_loss: 0.4749 - val_accuracy: 0.8619
Epoch 6/20
124/124 [==============================] - 1774s 14s/step - loss: 0.1029 - accuracy: 0.9675 - val_loss: 0.4317 - val_accuracy: 0.8790
Epoch 7/20
124/124 [==============================] - 1772s 14s/step - loss: 0.0804 - accuracy: 0.9773 - val_loss: 0.5289 - val_ac

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
124/124 [==============================] - 1796s 14s/step - loss: 0.7887 - accuracy: 0.6638 - val_loss: 0.7221 - val_accuracy: 0.7409
Epoch 2/20
124/124 [==============================] - 1852s 15s/step - loss: 0.4336 - accuracy: 0.8467 - val_loss: 0.4502 - val_accuracy: 0.8206
Epoch 3/20
124/124 [==============================] - 1823s 15s/step - loss: 0.2933 - accuracy: 0.9012 - val_loss: 0.4408 - val_accuracy: 0.8387
Epoch 4/20
124/124 [==============================] - 1840s 15s/step - loss: 0.2197 - accuracy: 0.9276 - val_loss: 0.3938 - val_accuracy: 0.8649
Epoch 5/20
124/124 [==============================] - 1873s 15s/step - loss: 0.1549 - accuracy: 0.9493 - val_loss: 0.4718 - val_accuracy: 0.8569
Epoch 6/20
124/124 [==============================] - 1796s 14s/step - loss: 0.1216 - accuracy: 0.9632 - val_loss: 0.4280 - val_accuracy: 0.8770
Epoch 7/20
124/124 [==============================] - 1863s 15s/step - loss: 0.0946 - accuracy: 0.9735 - val_loss: 0.6034 - val_ac

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
124/124 [==============================] - 1832s 15s/step - loss: 0.7405 - accuracy: 0.6925 - val_loss: 0.5273 - val_accuracy: 0.8095
Epoch 2/20
124/124 [==============================] - 1812s 15s/step - loss: 0.4100 - accuracy: 0.8558 - val_loss: 0.4333 - val_accuracy: 0.8387
Epoch 3/20
124/124 [==============================] - 1828s 15s/step - loss: 0.2566 - accuracy: 0.9165 - val_loss: 0.4226 - val_accuracy: 0.8639
Epoch 4/20
124/124 [==============================] - 1825s 15s/step - loss: 0.1760 - accuracy: 0.9430 - val_loss: 0.4115 - val_accuracy: 0.8831
Epoch 5/20
124/124 [==============================] - 1835s 15s/step - loss: 0.1272 - accuracy: 0.9584 - val_loss: 0.4183 - val_accuracy: 0.8800
Epoch 6/20
124/124 [==============================] - 1808s 15s/step - loss: 0.0890 - accuracy: 0.9710 - val_loss: 0.4772 - val_accuracy: 0.8790
Epoch 7/20
124/124 [==============================] - 1847s 15s/step - loss: 0.0804 - accuracy: 0.9745 - val_loss: 0.4780 - val_ac

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
124/124 [==============================] - 1843s 15s/step - loss: 0.7508 - accuracy: 0.6872 - val_loss: 0.4941 - val_accuracy: 0.8145
Epoch 2/20
124/124 [==============================] - 1873s 15s/step - loss: 0.4364 - accuracy: 0.8454 - val_loss: 0.3691 - val_accuracy: 0.8679
Epoch 3/20
124/124 [==============================] - 1850s 15s/step - loss: 0.2872 - accuracy: 0.9017 - val_loss: 0.3540 - val_accuracy: 0.8770
Epoch 4/20
124/124 [==============================] - 1876s 15s/step - loss: 0.1968 - accuracy: 0.9329 - val_loss: 0.3645 - val_accuracy: 0.8881
Epoch 5/20
124/124 [==============================] - 1831s 15s/step - loss: 0.1332 - accuracy: 0.9569 - val_loss: 0.3632 - val_accuracy: 0.8962
Epoch 6/20
124/124 [==============================] - 1847s 15s/step - loss: 0.0983 - accuracy: 0.9675 - val_loss: 0.4811 - val_accuracy: 0.8871
Epoch 7/20
124/124 [==============================] - 1840s 15s/step - loss: 0.0915 - accuracy: 0.9685 - val_loss: 0.4386 - val_ac

In [22]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(test_data, steps=validation_steps)

31/31 [==============================] - 113s 4s/step - loss: 0.6027 - accuracy: 0.9002


In [23]:
# Predict on the validation set
predictions = model.predict(test_data, steps=validation_steps)
y_pred = np.argmax(predictions.logits, axis=1)
y_true = test['Sentiment1'][:len(y_pred)]

31/31 [==============================] - 112s 4s/step


In [24]:
# Calculate average accuracy across all folds
average_accuracy = np.mean(accuracy_list)
print(f'Average Accuracy: {average_accuracy * 100:.2f}%\n')
report = classification_report(y_true, y_pred, target_names=label_encoder.classes_)
print(f'Fold {fold} Classification Report:\n{report}')

Average Accuracy: 90.02%

Fold 5 Classification Report:
              precision    recall  f1-score   support

     negatif       0.93      0.90      0.91       306
      netral       0.93      0.86      0.89       291
     positif       0.86      0.93      0.90       395

    accuracy                           0.90       992
   macro avg       0.91      0.90      0.90       992
weighted avg       0.90      0.90      0.90       992

